In [5]:
import numpy as np
import pandas as pd
from sodapy import Socrata
import datetime
import time

In [11]:
client = Socrata("data.cityofnewyork.us", None)
results = client.get_all("8vv7-7wx3")
emc_df = pd.DataFrame.from_records(results)

In [12]:
emc_df["email_body"]=emc_df["email_body"].apply(lambda x: np.NaN if x== "[blank]" else x)
emc_df = emc_df.dropna()
emc_df.duplicated(keep = "first")
emc_df['notificationtype'] = emc_df['notificationtype'].astype('category')
emc_df['date_and_time'] = pd.to_datetime(emc_df['date_and_time'])
emc_df['date_and_time'] = emc_df['date_and_time'].dt.date

Noticed that there are public health notifications in 2019.12 and trying to find out what are thoses notifications about

In [13]:
December = emc_df[emc_df["notificationtype"].apply(lambda x : x == "Public Health")]
December = December[December["notification_title"].apply(lambda x : x != "[blank]")]
start_t = "2019-11-01"
end_t = "2020-01-01"
start_t=datetime.date(*map(int,start_t.split('-')))
end_t=datetime.date(*map(int,end_t.split('-')))
December = December[December["date_and_time"].apply(lambda x : x > start_t and x < end_t)]
December

,date_and_time,email_body,notification_title,notificationtype,record_id
1420,2019-12-20,8AM: Today will be the last day of the recepti...,Nixle Messages - 12/20 (CW),Public Health,15628
1448,2019-12-18,8AM: Reception center open 12pm-6pm at 145-1...,Nixle Messages - 12/18,Public Health,15594
1563,2019-12-11,8AM: Reception center open 12pm-7pm Wed at 1...,Nixle Messages - 12/11 - Queens Sewage Inciden...,Public Health,15474
1596,2019-12-09,"8 AM, Reception center open 12pm-7pm Mon at ...",Nixle Messages - 12/9 - Queens Sewage Incident...,Public Health,15426
1606,2019-12-08,8 AM:- Reception center open 12pm-4pm Sun at ...,Nixle Messages - 12/8 - Queens Sewage Incident...,Public Health,15443
1607,2019-12-08,8 AM:- Reception center open 12pm-4pm Sun at ...,Nixle Messages - 12/8 - Queens Sewage Incident...,Public Health,15418
1618,2019-12-07,8 AM:- Out Reception center open 9am-3pm Sat ...,Nixle Messages - 12/7 - Queens Sewage Incident...,Public Health,15417


df_health: notification of type "Public Health" since there's confirmed cases(2019-02-29)

In [22]:
start_time='2020-02-28'
start_date=datetime.date(*map(int,start_time.split('-')))
df_start = emc_df[emc_df['date_and_time'].apply(lambda x: x > start_date)]
df_health = df_start[df_start["notificationtype"].apply(lambda x : x == "Public Health")]
print(df_health.shape[0])

117


Find notifications in “Public Health” type after break out and title includes covid-related keywords

In [24]:
words_list = ['COVID19','COVID-19','Coronavirus','COVID']
covid = df_health[df_health['notification_title'].str.contains('|'.join(words_list))]
covid
covid_percentage = "{:.2%}".format(covid.shape[0] / df_health.shape[0])
print(covid_percentage)

95.73%


In [27]:
covid_all = df_start[df_start['notification_title'].str.contains('|'.join(words_list))]
covid_all.loc[:,'notificationtype'].value_counts()
counts_covid = covid_all.groupby(['notificationtype'])['record_id'].count().reset_index(name='covid_count')
counts_covid = counts_covid.loc[lambda counts_covid: counts_covid['covid_count'] > 0]

Find notifications related to daily life or welfare in covid notifications

In [32]:
list2 = ['buy','shop','grocery','food','meal','free','delivery','price']
shop_body = covid[covid['email_body'].str.contains('|'.join(list2))]
shop_title = covid[covid['notification_title'].str.contains('|'.join(list2))]
merge = [shop_body, shop_title]
shop = pd.concat(merge)
shop.shape[0]

24

In [39]:
face = covid[covid['email_body'].str.contains('face')]
face

,date_and_time,email_body,notification_title,notificationtype,record_id
20,2020-05-07,Notification issued 05-07-2020 at 12:00 PM. ...,COVID19: Free Face Coverings,Public Health,17052
71,2020-05-02,Notification issued 05-02-2020 at 11:00 AM. ...,COVID-19: Social Distancing,Public Health,17001
80,2020-05-01,Notification issued 05-01-2020 at 10:40 AM. ...,COVID-19 - Ferries,Public Health,16994
83,2020-04-30,Cleanliness at home can help stop the spread o...,Targeted COVID-19 Notify - Multilingual Templa...,Public Health,16949
102,2020-04-28,"If you must leave your home, avoid crowded pla...",Targeted COVID-19 Notify - Multilingual Templa...,Public Health,16947
123,2020-04-26,The most common symptoms of COVID-19 are fever...,Targeted COVID-19 Notify - Multilingual Templa...,Public Health,16945
165,2020-04-20,REMINDER: All people must wear a face covering...,COVID-19: Face Coverings,Public Health,16904
182,2020-04-17,Notification issued 04-17-2020 at 3:00 PM. ...,COVID19 - Support Grocery Workers,Public Health,16886
197,2020-04-15,Notification issued 04-15-2020 at 5:03 PM. ...,COVID19 - Face Coverings,Public Health,16874
212,2020-04-13,o § (1/3) New Yorkers: Wear a face covering...,COVID-19 - Facial Covering/Testing,Public Health,16859


In [34]:
social_distance = covid[covid['email_body'].str.contains('social distanc')]
social_distance

,date_and_time,email_body,notification_title,notificationtype,record_id
9,2020-05-08,Notification issued 05-08-2020 at 12:00 PM. ...,COVID-19: Open Streets,Public Health,17053
165,2020-04-20,REMINDER: All people must wear a face covering...,COVID-19: Face Coverings,Public Health,16904
197,2020-04-15,Notification issued 04-15-2020 at 5:03 PM. ...,COVID19 - Face Coverings,Public Health,16874
284,2020-04-08,Notification issued 04-08-2020 at 9:02 AM. ...,COVID19 - NYC Parks Dogs,Public Health,16786
433,2020-03-21,Notification issued 03-21-2020 at 1:30 PM You...,Notification/Nixle (English/Spanish/Chinese) -...,Public Health,16634


Notifications about violence during epidemic.

In [37]:
violence = covid[covid['notification_title'].str.contains('Violence')]
violence

,date_and_time,email_body,notification_title,notificationtype,record_id
54,2020-05-04,Notification issued 05-04-2020 at 11:30 AM. ...,COVID-19: Domestic Violence Services,Public Health,17013
168,2020-04-20,"When home isnt safe, domestic violence surviv...",COVID-19: Domestic Violence Resources,Public Health,16903
217,2020-04-13,1/2 We understand home is not always safe. Vi...,COVID-19 - Domestic Violence,Public Health,16855
304,2020-04-06,Notification issued 03-06-2020 at 2:08 PM. ...,COVID-19 - Violence,Public Health,16764


Notification about helping the community during epidemic.

In [36]:
list3 = ['help','volunt','donation']
helping = covid[covid['email_body'].str.contains('|'.join(list3))]
helping

,date_and_time,email_body,notification_title,notificationtype,record_id
9,2020-05-08,Notification issued 05-08-2020 at 12:00 PM. ...,COVID-19: Open Streets,Public Health,17053
83,2020-04-30,Cleanliness at home can help stop the spread o...,Targeted COVID-19 Notify - Multilingual Templa...,Public Health,16949
84,2020-04-30,Notification issued 04-30-2020 at 2:00 PM. ...,COVID19 - Volunteer NYC,Public Health,16987
104,2020-04-28,Notification Issued 04-28-2020 at 12:00 PM. ...,COVID-19: NYC Well Hotline,Public Health,16971
138,2020-04-23,Notification issued 04-23-2020 at 3:00 PM. ...,COVID19 - Help you City,Public Health,16933
206,2020-04-14,Notification issued 04-14-2020 at 11:00 AM. ...,COVID-19: Plasma Donation,Public Health,16840
230,2020-04-12,The City is collecting voluntary information a...,COVID-19 - Collecting Information,Public Health,16839
251,2020-04-10,Notification issued 04-10-2020 at 12:00 PM ...,COVID19 - Organization Assistance,Public Health,16818
253,2020-04-10,Notification issued 04-10-2020 at 10:00 AM. ...,COVID19 - How to get involved,Public Health,16816
279,2020-04-08,Notification Issued 04-08-2020 at 12:00 PM ...,COVID19 - Coping,Public Health,16787
